In [1]:
!pip install --upgrade "google-cloud-aiplatform[langchain,reasoningengine]" cloudpickle==3.0.0 pydantic==2.7.4 langchain-google-community google-cloud-discoveryengine google-api-python-client requests

INFO: pip is looking at multiple versions of langchain-google-vertexai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-instrumentation to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.4 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.2
    Uninstalling pydantic_core-2.27.2:
      Successfully uninstalled pydantic_core-2.27.2
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 3.1.0
    Uninstalli

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [4]:
####Set Google Cloud project information and initialize Vertex AI SDK

In [1]:
PROJECT_ID = "qwiklabs-gcp-03-414f71a120f8"  
LOCATION = "us-central1"
STAGING_BUCKET = "gs://qwiklabs-gcp-03-414f71a120f8"  

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

In [5]:
####Import libraries and define generative model

In [2]:
from googleapiclient import discovery
from IPython.display import display, Markdown
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain_core import prompts
from langchain.memory import ChatMessageHistory
from vertexai.preview import reasoning_engines

In [3]:
model = "gemini-1.5-pro-001"

In [7]:
####Task 2. Create a Data Store in Vertex AI Search
"""
In this task, you'll create a Search App and Data Store within Vertex AI Search and index records from a movie data set.

In the Google Cloud Console, select Navigation menu > Agent Builder.

Read and agree to the Terms of Service, then click Continue and activate the API.

Click +CREATE APP.

On the Create App page, select the Search app type, enter the following details and click on Continue.
"""

"\nIn this task, you'll create a Search App and Data Store within Vertex AI Search and index records from a movie data set.\n\nIn the Google Cloud Console, select Navigation menu > Agent Builder.\n\nRead and agree to the Terms of Service, then click Continue and activate the API.\n\nClick +CREATE APP.\n\nOn the Create App page, select the Search app type, enter the following details and click on Continue.\n"

In [8]:
DATA_STORE_ID = "cymbal-movie-data_1737189767070"
LOCATION_ID = "global"

In [11]:
####Task 3. Define and test your agent locally
"""
In this task, construct a conversational AI agent equipped with reasoning abilities. The agent will utilize a tool function to interact with Vertex AI Search and leverage the Langchain Agent framework to combine the model, tools, and reasoning for effective user interaction.

Define Python functions as tools
This component of the agent employs Python functions as tools, allowing the Gemini model to interact with external systems, databases, and document repositories. This enables the model to access the latest information and perform actions within these systems.

Define a function that sends a query to Vertex AI Search and returns relevant records from the previously created data store.
"""

'\nIn this task, construct a conversational AI agent equipped with reasoning abilities. The agent will utilize a tool function to interact with Vertex AI Search and leverage the Langchain Agent framework to combine the model, tools, and reasoning for effective user interaction.\n\nDefine Python functions as tools\nThis component of the agent employs Python functions as tools, allowing the Gemini model to interact with external systems, databases, and document repositories. This enables the model to access the latest information and perform actions within these systems.\n\nDefine a function that sends a query to Vertex AI Search and returns relevant records from the previously created data store.\n'

In [12]:
def search_kaggle_movies(query: str) -> str:
    """Search across records in the Kaggle Movies data set."""
    from langchain_google_community import VertexAISearchRetriever

    retriever = VertexAISearchRetriever(
        project_id=PROJECT_ID,
        data_store_id=DATA_STORE_ID,
        location_id=LOCATION_ID,
        engine_data_type=1,
        max_documents=10,
    )

    result = str(retriever.invoke(query))
    return result

In [13]:
search_kaggle_movies("Harry Potter")

/opt/conda/lib/python3.10/site-packages/langchain_google_community/vertex_ai_search.py:364: UserWarning: Beta features are configured but beta=False. The following beta features will be ignored:['custom_embedding_ratio']
  warnings.warn(


'[]'

In [14]:
prompt = {
    "history": lambda x: x["history"],
    "input": lambda x: x["input"],
    "agent_scratchpad": (lambda x: format_to_tool_messages(x["intermediate_steps"])),
} | prompts.ChatPromptTemplate.from_messages(
    [
        prompts.MessagesPlaceholder(variable_name="history"),
        ("user", "{input}"),
        prompts.MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Initialize session history
store = {}


def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [15]:
agent = reasoning_engines.LangchainAgent(
    prompt=prompt,
    model=model,
    chat_history=get_session_history,
    model_kwargs={"temperature": 0},
    tools=[search_kaggle_movies],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [16]:
response = agent.query(
    input="List some sci-fi movies from the 1990s",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

/opt/conda/lib/python3.10/site-packages/langchain_google_community/vertex_ai_search.py:364: UserWarning: Beta features are configured but beta=False. The following beta features will be ignored:['custom_embedding_ratio']
  warnings.warn(


I was unable to find any sci-fi movies from the 1990s in the Kaggle Movies dataset. 


In [17]:
agent = reasoning_engines.LangchainAgent(
    prompt=prompt,
    model=model,
    chat_history=get_session_history,
    model_kwargs={"temperature": 0},
    tools=[search_kaggle_movies],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [ ]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[langchain,reasoningengine]",
        "cloudpickle==3.0.0",
        "pydantic==2.7.4",
        "langchain-google-community",
        "google-cloud-discoveryengine",
    ],
)

Creating bucket qwiklabs-gcp-03-414f71a120f8 in location='us-central1'
Writing to gs://qwiklabs-gcp-03-414f71a120f8/reasoning_engine/reasoning_engine.pkl
Writing to gs://qwiklabs-gcp-03-414f71a120f8/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://qwiklabs-gcp-03-414f71a120f8/reasoning_engine/dependencies.tar.gz
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/580799819376/locations/us-central1/reasoningEngines/6016580403748405248/operations/3117839939828449280


In [ ]:
service = discovery.build("cloudresourcemanager", "v1")
request = service.projects().get(projectId=PROJECT_ID)
response = request.execute()
project_number = response["projectNumber"]
project_number

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role=roles/discoveryengine.editor

In [ ]:
response = remote_agent.query(
    input="List some sci-fi movies from the 1990s",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

In [ ]:
response = remote_agent.query(
    input="Who are the actors in The Matrix?",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

In [ ]:
response = remote_agent.query(
    input="Are those actors in any other movies?",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))